# Analyzing Computer Science versus Business Management Introductory Course Professors Reviews and Their Trends Over Time

William Ingold, Erik Kelemen, Ashish Manda

## Introduction

## Grabbing Introductory Course Professors From UMD.io

In [ ]:
import requests
professors_url = "https://api.umd.io/v1/professors"

In [ ]:
def get_professors_for_courses(course_ids):
    professors = {}
    
    for course_id in course_ids:
        params = {'course_id': course_id}

        response = requests.get(professors_url, params)
        
        if response.status_code == 200:

            for item in response.json():
                name = item['name']

                if name in professors:
                    professors[name].add(course_id)
                else:
                    professors[name] = {course_id}

            return professors

### Computer Science Professors

In [ ]:
cmsc_course_ids = ["CMSC131", "CMSC132", "CMSC216", "CMSC250"]
cmsc_professors = get_professors_for_courses(cmsc_course_ids)

if not cmsc_professors:
    print("Error response from umd.io API")

cmsc_professors.pop('Iason Filippou') # A typo of Jason Filippou from the database
print(cmsc_professors)

### Business Management Professors

In [ ]:
bmgt_course_ids = ["BMGT110", "BMGT220", "BMGT221", "BMGT230"]
bmgt_professors = get_professors_for_courses(bmgt_course_ids)

if not bmgt_professors:
    print("Error response from umd.io API")

print(bmgt_professors)

## Grabbing Reviews

## From RateMyProfessor

In [ ]:
from bs4 import BeautifulSoup

ratemyprofessor_url = "https://www.ratemyprofessors.com/search.jsp"

params = {'queryoption':'HEADER', 'schoolID':'1270', 'queryBy':'teacherName', 'schoolName':'University+of+Maryland'}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Headers": "Content-Type",
    "Access-Control-Allow-Methods": "GET"
}

In [ ]:
def find_rmp_professor_url(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    partial_url = soup.find('li', class_='listing PROFESSOR').find('a', href=True)
    
    if partial_url:
        main_url = "https://www.ratemyprofessors.com"
        return main_url + partial_url['href']

In [ ]:
def query_rmp_for_professor_url(professor_name, headers, params):
    params['query'] = professor_name
    
    response = requests.get(ratemyprofessor_url, headers=headers, params=params)
    
    if response.status_code == 200:
        return find_rmp_professor_url(response.text)

In [ ]:
def get_prof_rmp_stats(page_text):
    soup = BeautifulSoup(page_text, 'html.parser')
    
    rating = soup.select('div[class*="RatingValue__Numerator"]')
    
    items = soup.select('div[class*="FeedbackItem__FeedbackNumber")]')
    return items

#### Need Selenium because Javascript hides more reviews

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Firefox(executable_path='./bin/geckodriver.exe')

In [ ]:
def load_all_rmp_reviews(page_url):
    driver.get(page_url)
    
    # RateMyProfessors has a cookies pop up that overlays the website, it needs to be closed first
    time.sleep(0.5)
    close_cookies = driver.find_elements(By.XPATH, '//button[text()="Close"]')
    
    if close_cookies:
        close_cookies[0].click()
        
    load_more = driver.find_elements(By.XPATH, '//button[text()="Load More Ratings"]')
    
    # RateMyProfessors paginates the reviews via Javascript, so we must continually load more while the button is available
    while load_more:
        load_more[0].click()
        time.sleep(1)
        load_more = driver.find_elements(By.XPATH, '//button[text()="Load More Ratings"]')
        
    return driver.find_element_by_id('ratingsList').text

In [ ]:
def parse_single_rmp_review(review_item):
    soup = BeautifulSoup(review_item, 'html.parser')
    course = soup.select('div[class*="RatingHeader__StyledClass"]').text
    print(course)
    

In [ ]:
def get_all_rmp_reviews(review_html):
    soup = BeautifulSoup(review_html, 'html.parser')
    return None

### Computer Science Professors

In [ ]:
fawzi_url = query_rmp_for_professor_url('Fawzi Emad', headers, params)
print(fawzi_url)

response = requests.get(fawzi_url)

Split for testing purposes, don't want to query the page multiple times

In [ ]:
print(response.text)

if response.status_code == 200:
    print(get_prof_rmp_stats(response.text))
else:
    print('error')

### Business Management Professors

## From Planetterp

### Computer Science Professors

### Business Management Professors